OR回路をNNで解く

In [1]:
import numpy as np
import chainer
from chainer import links as L
from chainer import initializer as I
from chainer import functions as F
from chainer import training
from chainer.training import extensions
print(chainer.__version__)

4.3.1


/home/osaka/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
class MyChain(chainer.Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(2, 3)    #入力層２、中間層３
            self.l2 = L.Linear(3, 2)    #中間層３，出力層２
    
    def __call__(self, x):
        h1 = F.relu(self.l1(x))    #ReLU function
        y = self.l2(h1)
        return y

epoch = 100
batch_size = 4

#データの作成
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_train = np.array([0, 1, 1, 1], dtype=np.int32)
train = chainer.datasets.TupleDataset(x_train, y_train)
test = chainer.datasets.TupleDataset(x_train, y_train)
print("train:", train)

#ニューラルネットワークの登録
model = L.Classifier(MyChain(), lossfun=F.softmax_cross_entropy)
#chainer.serializer.load_npz("results/out.model", model)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

#イテレータの定義
train_iter = chainer.iterators.SerialIterator(train, batch_size)    #学習用
test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)    #評価用

#アップデータの登録
updater = training.StandardUpdater(train_iter, optimizer)

#トレーナの保存
trainer = training.Trainer(updater, (epoch, "epoch"))

#学習状況の保存や表示
trainer.extend(extensions.LogReport())    #ログ
trainer.extend(extensions.Evaluator(test_iter, model))    #エポック数の表示
trainer.extend(extensions.PrintReport(["epoch", "main/loss", "validation/main/loss", 
                                       "main/accuracy", "validation/main/accuracy", "elapsed_time"]))    #計算状態の表示
#trainer.extend(extentions.dump_graph("main/loss"))    #NNの構造
#trainer.extend(extensions.PlotReport["main/loss", "validation/main/loss"], "epoch", file_name="loss.png")    #誤差のグラフ
#trainer.extend(extensions.PlotReport["main/accuracy", "validation/main/accuracy"], "epoch", file_name="accuracy.png")    #精度のグラフ
#trainer.extend(extensions.snapshot(), trigger=(100, "epoch"))    #学習再開のためのスナップショットの出力

#chainer.serializers.load_npz("result/snapshot_iter_500", trainer)    #学習再開用
#chainer.serializers.save_npz("result/out.model")

#学習開始
trainer.run()


train: <chainer.datasets.tuple_dataset.TupleDataset object at 0x7ff981f600b8>
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.693147    0.692648              0.25           0.75                      0.00337495    
2           0.692648    0.692149              0.75           0.75                      0.00698262    
3           0.692149    0.691652              0.75           0.75                      0.0497372     
4           0.691652    0.691155              0.75           0.75                      0.0623164     
5           0.691155    0.69066               0.75           0.75                      0.0696947     
6           0.69066     0.690166              0.75           0.75                      0.092315      
7           0.690166    0.689673              0.75           0.75                      0.101835      
8           0.689673    0.689181              0.75           0.75                      0.108736      
9     

パーセプトロンをNNで解く<br>
MyChainを変えればネットワークを改造できる

In [20]:
class MyChain(chainer.Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(2, 2)    #入力１，中間層１
            
    def __call__(self, x):
        y = self.l1(x)
        return y
    

epoch = 100
batch_size = 4

#データの作成
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_train = np.array([0, 1, 1, 1], dtype=np.int32)
train = chainer.datasets.TupleDataset(x_train, y_train)
test = chainer.datasets.TupleDataset(x_train, y_train)
print("train:", train)

#ニューラルネットワークの登録
model = L.Classifier(MyChain(), lossfun=F.softmax_cross_entropy)
#chainer.serializer.load_npz("results/out.model", model)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

#イテレータの定義
train_iter = chainer.iterators.SerialIterator(train, batch_size)    #学習用
test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)    #評価用

#アップデータの登録
updater = training.StandardUpdater(train_iter, optimizer)

#トレーナの保存
trainer = training.Trainer(updater, (epoch, "epoch"))

#学習状況の保存や表示
trainer.extend(extensions.LogReport())    #ログ
trainer.extend(extensions.Evaluator(test_iter, model))    #エポック数の表示
trainer.extend(extensions.PrintReport(["epoch", "main/loss", "validation/main/loss", 
                                       "main/accuracy", "validation/main/accuracy", "elapsed_time"]))    #計算状態の表示
#trainer.extend(extentions.dump_graph("main/loss"))    #NNの構造
#trainer.extend(extensions.PlotReport["main/loss", "validation/main/loss"], "epoch", file_name="loss.png")    #誤差のグラフ
#trainer.extend(extensions.PlotReport["main/accuracy", "validation/main/accuracy"], "epoch", file_name="accuracy.png")    #精度のグラフ
#trainer.extend(extensions.snapshot(), trigger=(100, "epoch"))    #学習再開のためのスナップショットの出力

#chainer.serializers.load_npz("result/snapshot_iter_500", trainer)    #学習再開用
#chainer.serializers.save_npz("result/out.model")

#学習開始
trainer.run()


train: <chainer.datasets.tuple_dataset.TupleDataset object at 0x7ff981f4f400>
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.790447    0.788727              0.25           0                         0.00317841    
2           0.788727    0.787011              0              0                         0.00885194    
3           0.787011    0.7853                0              0                         0.051841      
4           0.7853      0.783594              0              0                         0.0594085     
5           0.783594    0.781893              0              0                         0.0649229     
6           0.781893    0.780196              0              0                         0.0700113     
7           0.780196    0.778504              0              0.25                      0.0739901     
8           0.778504    0.776817              0.25           0.25                      0.0828812     
9     

98          0.650015    0.648851              0.75           0.75                      0.407292      
99          0.648851    0.647691              0.75           0.75                      0.413302      
100         0.647691    0.646537              0.75           0.75                      0.419818      


## NNによる手書き文字認識　By　Chainer

In [31]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

class MyChain(chainer.Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(64, 100)    #入力６４、中間層１００
            self.l2 = L.Linear(100, 100)    #中間１００，中間１００
            self.l3 = L.Linear(100, 10)    #中間層１００、出力層１０
            
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y
    
epoch = 20
batch_size = 100

#データの作成
digits = load_digits()
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2)
x_train = x_train.astype(np.float32) / 17
x_test = x_test.astype(np.float32) / 17
train = chainer.datasets.TupleDataset(x_train, y_train)
test = chainer.datasets.TupleDataset(x_test, y_test)

print(x_train)

#ニューラルネットワークの登録
model = L.Classifier(MyChain(), lossfun=F.softmax_cross_entropy)
#chainer.serializer.load_npz("results/out.model", model)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

#イテレータの定義
train_iter = chainer.iterators.SerialIterator(train, batch_size)    #学習用
test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)    #評価用

#アップデータの登録
updater = training.StandardUpdater(train_iter, optimizer)

#トレーナの保存
trainer = training.Trainer(updater, (epoch, "epoch"))

#学習状況の保存や表示
trainer.extend(extensions.LogReport())    #ログ
trainer.extend(extensions.Evaluator(test_iter, model))    #エポック数の表示
trainer.extend(extensions.PrintReport(["epoch", "main/loss", "validation/main/loss", 
                                       "main/accuracy", "validation/main/accuracy", "elapsed_time"]))    #計算状態の表示
#trainer.extend(extentions.dump_graph("main/loss"))    #NNの構造
#trainer.extend(extensions.PlotReport["main/loss", "validation/main/loss"], "epoch", file_name="loss.png")    #誤差のグラフ
#trainer.extend(extensions.PlotReport["main/accuracy", "validation/main/accuracy"], "epoch", file_name="accuracy.png")    #精度のグラフ
#trainer.extend(extensions.snapshot(), trigger=(100, "epoch"))    #学習再開のためのスナップショットの出力

#chainer.serializers.load_npz("result/snapshot_iter_500", trainer)    #学習再開用
#chainer.serializers.save_npz("result/out.model")

#学習開始
trainer.run()


[[0.         0.23529412 0.5882353  ... 0.         0.         0.        ]
 [0.         0.         0.23529412 ... 0.05882353 0.         0.        ]
 [0.         0.         0.5882353  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.3529412  ... 0.7058824  0.         0.        ]
 [0.         0.         0.23529412 ... 0.3529412  0.         0.        ]
 [0.         0.         0.23529412 ... 0.05882353 0.         0.        ]]
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           2.16851     1.99049               0.354          0.549167                  0.0442737     
2           1.8061      1.5872                0.656429       0.723333                  0.0856669     
3           1.33202     1.07138               0.796667       0.8075                    0.130251      
4           0.890751    0.68953               0.852857       0.873333                  0.171197      


/home/osaka/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/chainer/optimizers/adam.py:111: RuntimeWarning: invalid value encountered in sqrt
  param.data -= hp.eta * (self.lr * m / (numpy.sqrt(vhat) + hp.eps) +


5           0.585392    0.468361              0.879286       0.900833                  0.223074      
6           0.416338    0.33789               0.903333       0.933333                  0.273515      
7           0.322727    0.279738              0.926429       0.943333                  0.314577      
8           0.261039    0.228505              0.937857       0.940833                  0.355586      
9           0.223129    0.194014              0.953333       0.959167                  0.39916       
10          0.190313    0.177999              0.955          0.955                     0.446948      
11          0.166978    0.15626               0.964          0.9625                    0.493359      
12          0.157149    0.142209              0.969286       0.973333                  0.534672      
13          0.140012    0.133418              0.975714       0.966667                  0.575576      
14          0.128455    0.125054              0.972          0.973333             

## CNNの実行
↑との違いはネットワーク構造と学習データの違いだけ

In [51]:
class MyChain(chainer.Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            #１層目の畳み込み層（入力チャンネル数（RGBなど）、出力チャンネル数、フィルタサイズ、ストライドサイズ、パディングサイズ）
            self.conv1 = L.Convolution2D(1, 16, 3, stride=1, pad=1)
            self.conv2 = L.Convolution2D(16, 64, 3, stride=1, pad=1)    #２層目の畳み込み層
            self.l3 = L.Linear(256, 10)    #出力層。分類用
            
    def __call__(self, x):
        #max_pooling_2d(入力データ、フィルタサイズ、ストライドサイズ)
        h1 = F.max_pooling_2d(F.relu(self.conv1(x)), ksize=2, stride=2)    #最大値プーリングは２×２、活性化関数にReLU
        h2 = F.max_pooling_2d(F.relu(self.conv2(h1)), ksize=2, stride=2)
        y = self.l3(h2)
        return y
    
epoch = 20
batch_size = 100

#データの作成
digits = load_digits()
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2)
#print("original:", x_train)

x_train = x_train.reshape((len(x_train), 1, 8, 8))    #1*8*8の行列に整形
x_test = x_test.reshape((len(x_test), 1, 8, 8))

x_train = (x_train).astype(np.float32)
x_test = (x_test).astype(np.float32)
train = chainer.datasets.TupleDataset(x_train, y_train)
test = chainer.datasets.TupleDataset(x_test, y_test)



#ニューラルネットワークの登録
model = L.Classifier(MyChain(), lossfun=F.softmax_cross_entropy)
#chainer.serializer.load_npz("results/out.model", model)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

#イテレータの定義
train_iter = chainer.iterators.SerialIterator(train, batch_size)    #学習用
test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)    #評価用

#アップデータの登録
updater = training.StandardUpdater(train_iter, optimizer)

#トレーナの保存
trainer = training.Trainer(updater, (epoch, "epoch"))

#学習状況の保存や表示
trainer.extend(extensions.LogReport())    #ログ
trainer.extend(extensions.Evaluator(test_iter, model))    #エポック数の表示
trainer.extend(extensions.PrintReport(["epoch", "main/loss", "validation/main/loss", 
                                       "main/accuracy", "validation/main/accuracy", "elapsed_time"]))    #計算状態の表示
#trainer.extend(extentions.dump_graph("main/loss"))    #NNの構造
#trainer.extend(extensions.PlotReport["main/loss", "validation/main/loss"], "epoch", file_name="loss.png")    #誤差のグラフ
#trainer.extend(extensions.PlotReport["main/accuracy", "validation/main/accuracy"], "epoch", file_name="accuracy.png")    #精度のグラフ
#trainer.extend(extensions.snapshot(), trigger=(100, "epoch"))    #学習再開のためのスナップショットの出力

#chainer.serializers.load_npz("result/snapshot_iter_500", trainer)    #学習再開用
#chainer.serializers.save_npz("result/out.model")

#学習開始
trainer.run()


/home/osaka/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/chainer/optimizers/adam.py:111: RuntimeWarning: invalid value encountered in sqrt
  param.data -= hp.eta * (self.lr * m / (numpy.sqrt(vhat) + hp.eps) +


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           3.90026     1.55917               0.227333       0.473333                  0.381306      
2           1.1175      0.731706              0.652143       0.8                       0.675543      
3           0.530927    0.466744              0.839333       0.858333                  1.05082       
4           0.339363    0.338975              0.896429       0.890833                  1.35138       
5           0.247055    0.288681              0.927857       0.901667                  1.64578       
6           0.198611    0.22781               0.946667       0.936667                  1.9644        
7           0.151786    0.210724              0.964286       0.935833                  2.25859       
8           0.129004    0.183146              0.965714       0.95                      2.56229       
9           0.108717    0.168174              0.974667       0.955833               